In [1]:
pip install tensorflow keras numpy pandas matplotlib scikit-learn opencv-python

In [2]:
pip install seaborn

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
import kagglehub

path = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'covid19-radiography-database' dataset.
Path to dataset files: /kaggle/input/covid19-radiography-database


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_dir = "/root/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5"

base_dir = "dataset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

categories = []


In [8]:
import os

dataset_dir = "/root/.cache/kagglehub/datasets/tawsifurrahman/covid19-radiography-database/versions/5"

for root, dirs, files in os.walk(dataset_dir):
    if any(f.lower().endswith(('.png', '.jpg', '.jpeg')) for f in files):
        print(f" Found images in: {root}")
        print(f" Example files: {files[:3]}")
        break


In [10]:
import os
import shutil
from sklearn.model_selection import train_test_split


dataset_dir = "/kaggle/input/covid19-radiography-database"
source_base_dir = os.path.join(dataset_dir, "COVID-19_Radiography_Dataset")

train_dir = "dataset/train"
test_dir = "dataset/test"

if os.path.exists("dataset"):
    shutil.rmtree("dataset")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

categories = [d for d in os.listdir(source_base_dir) if os.path.isdir(os.path.join(source_base_dir, d))]

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

for category in categories:

    source_dir = os.path.join(source_base_dir, category, "images")

    if not os.path.exists(source_dir):
        print(f" Folder not found: {source_dir}")
        continue

    images = [f for f in os.listdir(source_dir)
              if os.path.isfile(os.path.join(source_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if images:

        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        for img in train_images:
            shutil.copy(os.path.join(source_dir, img), os.path.join(train_dir, category, img))

        for img in test_images:
            shutil.copy(os.path.join(source_dir, img), os.path.join(test_dir, category, img))

        print(f" {category}: {len(train_images)} train | {len(test_images)} test images copied.")
    else:
        print(f" No images found in {source_dir}")

print("\n Dataset successfully split into training and testing folders!")

 Normal: 8153 train | 2039 test images copied.
 Lung_Opacity: 4809 train | 1203 test images copied.
 Viral Pneumonia: 1076 train | 269 test images copied.
 COVID: 2892 train | 724 test images copied.

 Dataset successfully split into training and testing folders!


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
import numpy as np

train_dir = 'dataset/train'
test_dir = 'dataset/test'


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(4, activation='softmax')(x)  # 4 classes

model = Model(inputs=base_model.input, outputs=predictions)

classes = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(classes),
    y=classes
)
class_weights = dict(enumerate(class_weights))
print("\nClass Weights:", class_weights)

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    class_weight=class_weights,
    verbose=1
)


model.save("covid_resnet50_finetuned_model.h5")
print("\n Fine-tuned model training complete and saved as covid_resnet50_finetuned_model.h5!")


Found 16930 images belonging to 4 classes.
Found 4235 images belonging to 4 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Class Weights: {0: np.float64(1.4635200553250345), 1: np.float64(0.880120607194843), 2: np.float64(0.5191340610818104), 3: np.float64(3.933550185873606)}


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 336s 569ms/step - accuracy: 0.4062 - loss: 1.1718 - val_accuracy: 0.6215 - val_loss: 0.9097
Epoch 2/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 277s 522ms/step - accuracy: 0.5874 - loss: 0.8366 - val_accuracy: 0.6090 - val_loss: 1.0209
Epoch 3/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 275s 518ms/step - accuracy: 0.6229 - loss: 0.7718 - val_accuracy: 0.4151 - val_loss: 1.8189
Epoch 4/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 274s 516ms/step - accuracy: 0.6420 - loss: 0.7353 - val_accuracy: 0.7124 - val_loss: 0.6903
Epoch 5/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 272s 513ms/step - accuracy: 0.6714 - loss: 0.6958 - val_accuracy: 0.6524 - val_loss: 0.8473
Epoch 6/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 273s 515ms/step - accuracy: 0.6907 - loss: 0.6559 - val_accuracy: 0.6800 - val_loss: 0.8167
Epoch 7/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 277s 522ms/step - accuracy: 0.6890 - loss: 0.6499 - val_accuracy: 0.6708 - val_loss: 0.8026
Epoch 8/25
530/530 ━━━━━━━━━━━━━━━━━━━━ 275s 520ms/step - accuracy: 0.7004 -